<a href="https://colab.research.google.com/github/toshif/colab1/blob/main/simple_haiku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/deepmind/dm-haiku

  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-7codpdzr
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-7codpdzr
  Created wheel for dm-haiku: filename=dm_haiku-0.0.6.dev0-py3-none-any.whl size=537308 sha256=3f8d0226978563449ba0073f97a08d57739ecc9b57511e74cc6476502aa4fd77
  Stored in directory: /tmp/pip-ephem-wheel-cache-p63y6fkg/wheels/06/28/69/ebaac5b2435641427299f29d88d005fb4e2627f4a108f0bdbc
Successfully built dm-haiku


In [2]:
!pip install optax

     |████████████████████████████████| 118 kB 4.2 MB/s 
     |████████████████████████████████| 57 kB 4.3 MB/s 


In [3]:
import functools
import math
from typing import Tuple, TypeVar
import warnings

import haiku as hk
import jax
import jax.numpy as jnp
import optax
import numpy as np
import pandas as pd
import plotnine as gg

T = TypeVar('T')
Pair = Tuple[T, T]

gg.theme_set(gg.theme_bw())
warnings.filterwarnings('ignore')

In [15]:
x = jnp.ones([3, 2, 4])
y = jnp.ones([3, 8]) * 5

def my_net_fn(x):
  mlp = hk.Sequential([
      hk.Flatten(),
      hk.Linear(8)
  ])
  return mlp(x)

model = hk.transform(my_net_fn)

rng = jax.random.PRNGKey(428)
params = model.init(rng, x)

model.apply(params, None, x)

DeviceArray([[ 0.37775657, -0.34618992, -0.22275901,  1.3984423 ,
               1.8037784 , -1.075372  , -1.9799409 , -0.06570727],
             [ 0.37775657, -0.34618992, -0.22275901,  1.3984423 ,
               1.8037784 , -1.075372  , -1.9799409 , -0.06570727],
             [ 0.37775657, -0.34618992, -0.22275901,  1.3984423 ,
               1.8037784 , -1.075372  , -1.9799409 , -0.06570727]],            dtype=float32)

In [16]:
@jax.jit
def loss(params, x, y):
  print(f'params={params}, x={x.shape}, y={y.shape}')
  pred = model.apply(params, None, x)
  return jnp.mean(jnp.square(pred - y))

@jax.jit
def update(params, opt_state, x, y):
  l, grads = jax.value_and_grad(loss)(params, x, y)
  grads, opt_state = opt.update(grads, opt_state)
  params = optax.apply_updates(params, grads)
  return l, params, opt_state  

opt = optax.adam(1e-3)
opt_state = opt.init(params)

loss(params, x, y)

params=FlatMap({
  'linear': FlatMap({
              'b': Traced<ShapedArray(float32[8])>with<DynamicJaxprTrace(level=0/1)>,
              'w': Traced<ShapedArray(float32[8,8])>with<DynamicJaxprTrace(level=0/1)>,
            }),
}), x=(3, 2, 4), y=(3, 8)


DeviceArray(26.462784, dtype=float32)

In [19]:
# training

for step in range(2000):
  train_loss, params, opt_state = update(params, opt_state, x, y)
  if step % 100 == 0:
    print(f'train_loss={train_loss}')

train_loss=17.805282592773438
train_loss=12.06995964050293
train_loss=7.966765880584717
train_loss=5.119480133056641
train_loss=3.2017714977264404
train_loss=1.9475939273834229
train_loss=1.1514157056808472
train_loss=0.6611995697021484
train_loss=0.36863744258880615
train_loss=0.19943076372146606
train_loss=0.10459356755018234
train_loss=0.053097523748874664
train_loss=0.026035308837890625
train_loss=0.012297021225094795
train_loss=0.005577513482421637
train_loss=0.00242120074108243
train_loss=0.0010025068186223507
train_loss=0.0003945300995837897
train_loss=0.00014704356726724654
train_loss=5.1715825975406915e-05


In [20]:
model.apply(params, None, x)

DeviceArray([[4.999982 , 4.9997377, 4.9998274, 4.9999914, 4.9999933,
              4.998029 , 4.9884763, 4.999901 ],
             [4.999982 , 4.9997377, 4.9998274, 4.9999914, 4.9999933,
              4.998029 , 4.9884763, 4.999901 ],
             [4.999982 , 4.9997377, 4.9998274, 4.9999914, 4.9999933,
              4.998029 , 4.9884763, 4.999901 ]], dtype=float32)